# Realize the Dropout regularization operation

In [33]:
import torch
from torch import nn
from d2l import torch as d2l

# define the dropout_layer function
def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1
    if dropout == 1:
        return torch.zeros_like(X)
    if dropout == 0:
        return X
    # what is this mask?
    mask = (torch.Tensor(X.shape).uniform_(0, 1) > dropout).float()
    return mask * X / (1.0 - dropout) # make some unit 0 and some units (1-dropout)

X = torch.tensor([1, 2, 3])
dropout = 0.5
(torch.Tensor(X.shape).uniform_(0 ,1) > dropout).float() # get the random value and generate the bool matrix
# the bool value can use the .float() function to convert into numeric value 
# the multiple arithemetic is much more faster than the select function

tensor([1., 0., 0.])

In [52]:
class Accumulator:
    """accumulate the value on 'n' parameters"""
    def __init__(self, n):
        self.data = [0.0] * n
    
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    
    def reset(self):
        self.data = [0.0] * len(self.data) # reset data to 0
        
    def __getitem__(self, idx):
        return self.data[idx]

def accuracy(y_hat, y):
    """calculate the numbers of correction"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        # the dimension must > 2 so the type > 2
        y_hat = y_hat.argmax(axis=1) # get the max partition as the type for every class matrix
    cmp = y_hat.type(y.dtype) == y # convert the same data type to compare the value
    return float(cmp.type(y.dtype).sum()) # just convert the type

def evaluate_accuracy(net, data_iter):
    """calculate the accuracy in special model"""
    if isinstance(net, torch.nn.Module):
        net.eval() # set the model into evalution model, this will not calculate the grad, can simplify our module
        
    metric = Accumulator(2) # this is a class self define, [0] is right number [1] is all 
    for X, y in data_iter:
        metric.add(accuracy(net(X), y), y.numel()) # net(X) can calculate the y
    return metric[0] / metric[1]

        

def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):
    """Define the final trainning function just use above items"""
#     animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 1], legend=['train loss', 'train acc', 'test acc'])
    
    # if we just want to malloc the list inital, we can use [] * num
    res_list = [[]] * num_epochs
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
#         animator.add(epoch + 1, train_metrics + (test_acc, ))
        
        res_list[epoch] = ([train_metrics[0] ,train_metrics[1], test_acc])
    train_loss, train_acc = train_metrics
    
    return  res_list


def train_epoch_ch3(net, train_iter, loss, updater):
    # we can get the basic blockes through these parameters, the net, the loss, the optimizer.
    if isinstance(net, torch.nn.Module):
        net.train()
        
    metric = Accumulator(3)
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y) # evalute the loss value, just need the true and the prediction, there is the cross
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad() # elimate the grad
            l.backward() # calculate the grad
            # actually the l.backward calculate the grad for upader to modify its parameters
            
            updater.step() # update the trainner
            metric.add(
                float(l) * len(y), accuracy(y_hat, y), 
                y.size().numel()
            )
        else:
            l.sum().backward()
            updater(X.shape[0]) # grad down by the X.shape[0] numbers
            metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    return metric[0] / metric[2], metric[1] / metric[2] 

In [53]:
# test the dropout_layer function
X = torch.arange(16, dtype=torch.float32).reshape((2, 8))
print(X)
print(dropout_layer(X, 0.0))
print(dropout_layer(X, 1.0))
print(dropout_layer(X, 0.5))

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[ 0.,  0.,  0.,  6.,  8., 10., 12.,  0.],
        [ 0., 18.,  0., 22., 24.,  0.,  0., 30.]])


In [54]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

dropout1, dropout2 = 0.2, 0.5

class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs, num_hiddens1, num_hiddens2, 
                 is_training=True):
        # just the training process need the dropout regularization, so the is_training is very useful
        super(Net, self).__init__()
        self.num_inputs = num_inputs
        self.training = is_training
        self.lin1 = nn.Linear(num_inputs, num_hiddens1)
        self.lin2 = nn.Linear(num_hiddens1, num_hiddens2)
        self.lin3 = nn.Linear(num_hiddens2, num_outputs)
        self.relu = nn.ReLU() # need a non-linear activation function

    def forward(self, X):
        H1 = self.relu(self.lin1(X.reshape((-1, self.num_inputs)))) # this is the single layer
        if self.training == True:
            H1 = dropout_layer(H1, dropout1) # use dropout just during the training process
        H2 = self.relu(self.lin2(H1))
        if self.training == True:
            H2 = dropout_layer(H2, dropout2)
        out = self.lin3(H2) # the final layer didn't need the relu activation function
        return out

net = Net(num_inputs, num_outputs, num_hiddens1, num_hiddens2, True)

In [55]:
num_epochs, lr, batch_size = 10, 0.5, 256
loss = nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
trainer = torch.optim.SGD(net.parameters(), lr = lr)
train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

[[0.9273614109039306, 0.6571333333333333, 0.8064],
 [0.5372532222747802, 0.8046333333333333, 0.7885],
 [0.46343474979400634, 0.8300833333333333, 0.8404],
 [0.4311900212605794, 0.84235, 0.8364],
 [0.4032404393831889, 0.8518166666666667, 0.833],
 [0.38453372139930725, 0.85805, 0.835],
 [0.37162484475771584, 0.86445, 0.8289],
 [0.3563654223759969, 0.8682333333333333, 0.8559],
 [0.3479771459420522, 0.8705, 0.8678],
 [0.33854454897244773, 0.8754166666666666, 0.8594]]